In [1]:
# Install required packages
!pip install langchain langchain-community langchain-core langchain-openai
!pip install pypdf faiss-cpu
!pip install ollama

In [2]:
from langchain.llms import Ollama
from langchain.tools import Tool
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

In [3]:
# Load the PDF document
pdf_path = "/content/VisioWise Thesis(1).pdf"
pdf_loader = PyPDFLoader(pdf_path)
pdf_docs = pdf_loader.load()

In [4]:
# Load the Cleveland Clinic webpage
cleveland_url = "https://my.clevelandclinic.org/health/diagnostics/10228-chest-x-ray"
web_loader = WebBaseLoader(cleveland_url)
web_docs = web_loader.load()

In [5]:
# Load a Wikipedia page (e.g., Chest X-ray)
wiki_url = "https://en.wikipedia.org/wiki/Chest_radiograph"
wiki_loader = WebBaseLoader(wiki_url)
wiki_docs = wiki_loader.load()

In [6]:
# Combine all documents
tools = pdf_docs + web_docs + wiki_docs

In [7]:
# Split documents into chunks for RAG
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(tools)

In [8]:
# Initialize embeddings (Hugging Face model for embeddings)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create a FAISS vector store
vector_store = FAISS.from_documents(chunks, embeddings)

# Set up the retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

<ipython-input-8-1ebc5392a8dc>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

In [20]:
# Initialize Ollama
ollama = Ollama(model="llama3.2", base_url="http://localhost:11434")

In [21]:
# Define custom prompt template
prompt_template = """You are a medical assistant specializing in chest X-ray analysis. Use the provided context to answer the question concisely and accurately. Cite the source (e.g., PDF page, URL) where relevant. If unsure, say so.

Context: {context}

Question: {question}

Answer: """
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "input"])

In [22]:
# Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=ollama,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [24]:
query = "What is VisioWise and how does it help with chest X-ray analysis?"
response = qa_chain({"query": query})
print(response["result"])

ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7b7b207c0b90>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [17]:
# Print answer and sources
print("Answer:", response["result"])
print("\nSources:")
for doc in response["source_documents"]:
    print(doc.metadata)

NameError: name 'response' is not defined

In [18]:
# Interactive loop for additional queries
while True:
    user_query = input("\nEnter your question (or 'quit' to exit): ")
    if user_query.lower() == 'quit':
        break
    response = qa_chain({"query": user_query})
    print("Answer:", response["result"])
    print("\nSources:")
    for doc in response["source_documents"]:
        print(doc.metadata)


Enter your question (or 'quit' to exit): ygi


ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7b7b207865d0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:

# Pull the specified Ollama model
!ollama pull llama3.1